# Unit 9: LightFM

You almost made it - this is the final lesson and it is also going to be the easiest one.

As you might assume there are a lot of recommender packages in Python out there. And yes, there are. In this lesson we will look at LightFM - an easy to user and lightweight implementation of different algorithms for factorization machines to implement CF, CBF and hybrid recommenders.

Within a few lines of code we set-up, train and use a recommender for recommendations.

* [LightFM on GitHub](https://github.com/lyst/lightfm)
* [LightFM documentation](https://making.lyst.com/lightfm/docs/home.html)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k
from lightfm import LightFM

## Load Data

In [ ]:
data = fetch_movielens(min_rating=4.0)

In [ ]:
data

## Collaborative Filtering

In [ ]:
params = {
    'no_components': 16,
    'loss': 'bpr',
    'learning_rate': 0.05,
    'random_state': 42 
}

epochs = 10

N = 10

In [ ]:
cf_model = LightFM(**params)

In [ ]:
data['train']

In [ ]:
cf_model.fit(data['train'], epochs=epochs, verbose=True)

In [ ]:
# if we provide training data with evaluation, known positives will be removed
prec_at_N = precision_at_k(cf_model, data['test'], data['train'], k=N)

In [ ]:
prec_at_N.mean()

## Hybrid (CF + CBF)

In [ ]:
hybrid_model = LightFM(**params)

In [ ]:
hybrid_model.fit(data['train'], item_features=data['item_features'],
                 epochs=epochs, verbose=True)

In [ ]:
prec_at_N = precision_at_k(hybrid_model, data['test'], data['train'], k=N,
                           item_features=data['item_features'])

In [ ]:
prec_at_N.mean()